<a href="https://colab.research.google.com/github/gabi1771/data_science_projects/blob/main/RAG_trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<h1>
<h1>APM 53674: ALTeGraD</h1>
<h2>Lab Session 4: Distillation and Retrieval Augmented Generation</h2>
<h4>Lecture: Dr. Guokan Shang<br>
Lab: Yang Zhang and Xiao Fei</h4>
<h5>Tuesday, November 18, 2025</h5>
<br>
</center>

<hr style="border:10px solid gray"> </hr>
<p style="text-align: justify;">
This handout includes theoretical introductions, <font color='blue'>coding tasks</font> and <font color='red'>questions</font>. Before the deadline, you should submit <a href='https://forms.gle/9dyaes6dimfvyjwq6' target="_blank">here</a> a <B>.ipynb</B> file named <b>Lastname_Firstname.ipynb</b> containing your notebook (with the gaps filled and your answers to the questions). Your answers should be well constructed and well justified. They should not repeat the question or generalities in the handout. When relevant, you are welcome to include figures, equations and tables derived from your own computations, theoretical proofs or qualitative explanations. One submission is required for each student. The deadline for this lab is <b>Novemver 23
, 2025 11:59 PM</b>. No extension will be granted. Late policy is as follows: ]0, 24] hours late → -5 pts; ]24, 48] hours late → -10 pts; > 48 hours late → not graded (zero).
</p>
<hr style="border:5px solid gray"> </hr>

# Install Requirements

In [ ]:
# Install required dependencies
!pip -q install torch tqdm jsonlines h5py
!pip -q install --upgrade transformers accelerate vllm
!pip install jedi
# !pip -q install datasets==2.21.0 pandas==2.2.2
# !pip -q install chromadb==0.4.22
# !pip -q install "numpy<2.0" --force-reinstall

# Part 1 - Model Distillation

> In this section, you’ll learn the difference between **white-box** and **black-box** distillation, generate **synthetic data** to train a **student model**, and implement **white-box distillation** to specialize a model for a **RAG on Wikipedia** use case.


<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 1:</br>
Explain briefly the difference between black-box and white-box distillation? </br> What are the advantages and inconvenients of each approach?
<hr style="border:10px solid red"> </hr>  
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 1: </b><br>
Your answer here.
<hr style="border:10px solid green"> </hr>
</font></h4>

### **Black-box distillation**

Black box distillation aims to learn from another model only from its output probability. We teach the student to reproduce these probabilities from the inputs.

**Adavantages** :
- **Simple to implement**: Query the teacher and train on its outputs. We don't need label at all as it is generated by the trained model itself.
- **No knowledge needed on the model**: Doesn't require access to teacher's architecture or weights.
- **Cross-architecture**: Can distill from any teacher model type to any student mdoel (Transformer to CNN for example).

**Inconvenients** :
- **Few information transfer**: Even if we take advantage of another model to train our (no need to have big dataset), we only learns from final outputs, missing rich intermediate representations.
- **Limited guidance**: Can't leverage layer-wise or attention-based knowledge.


### **White-box distillation**

White box distillation allows a student model to learn from a teacher model that we fully knows (weigths, architecture, etc). Here, we want to teach a small teacher model, some characteristics of the bigger teacher model. For example, we create a student model with 3 layers (teacher with 5). Then, try to make the first student layer to learn the first teacher layer, the second student layer to learn the third teacher layer, then 3->5.

**Adavantages** :
- **Rich knowledge transfer**: Can match intermediate representations, not just final outputs.
- **Efficient learning**: Student learns from multiple levels of abstraction simultaneously.
- **Targeted distillation**: Can focus on specific components (attention, embeddings, specific layers).


**Inconvenients** :
- **Requires access**: Needs the teacher's weights and architecture.
- **Architecture constraints**: Student and teacher often need compatible structures.
- **High computational cost during distillation**: Must compute and match multiple representations.

<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 2:</br>
What is the main requirement for a teacher/student pair of models to perform white-box distillation?
<hr style="border:10px solid red"> </hr>  
</font></h4>



<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 2: </b><br>
Your answer here.
<hr style="border:10px solid green"> </hr>
</font></h4>

With white box distillation, we aim to reproduce the behavior of several component of the teacher model. In order to do it, we must have **architecture compatibility** in order to have some logic on the way the pattern are learned in the student model. We must be a way to map corresponding components between teacher and student.


## 1.1 - Synthetic Data Generation

We're going to specialize a small 0.5B parameter model to perform RAG by distilling the abilities of a 7B parameter one.

For that we'll be using `Qwen/Qwen2.5-0.5B-Instruct` as student and `Qwen/Qwen2.5-7B-Instruct-AWQ` (quantized version of `Qwen2.5-7B-Instruct`) as teacher.  

In order to perform white-box distillation on generated answers we have two choices.

1. We can perform a forward pass with the teacher, a forward pass with student on the complete sequence, and backprop difference of logprobs using KL Loss.
2. Generation of samples with the teacher, save the logprobs and perform finetuning in a second step.

<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 3:</br>
What are the computational advantages of 1. vs 2.?
<hr style="border:10px solid red"> </hr>  
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 3: </b><br>
Your answer here.
<hr style="border:10px solid green"> </hr>
</font></h4>

The computational advantages of 1. is that we do not need to store a lot of data as we will only store the teacher and student response to a unique forward pass. The downside of it is that we have to wait for the inference of the teacher for each training step of the student, that can add a **huge amount of time especially** as the teacher is a bigger model than the student.

In the same time, 2. has a clear downside. We need to **store a lot of samples from the teacher in memory**, that we will then use to fine the student. This approach is **way more computationally effective** as, once we have infer a lot of samples with the teacher, we only have to train the student by one student forward pass per step. This is a lot quicker than the 1. Moreover, the s**amples we have from the teacher can be reused** whenever we want.

To conclude, we will prefer 1. if we have limited memory and 2. if we have a lot of it.


We're going to generate a bunch of questions related to wikipedia paragraphs.
For that we need to establish a system prompt that will allow for easy extraction.

In [ ]:
system_prompt = """
You are a question generator.
The user will provide:

```json
{"title": "the title of an article", "paragraph": "a paragraph from that article"}
```

Your task:

* Generate one clear, self-contained question that can be answered using only the provided paragraph.
* The question must be **specific**, **unambiguous**, and directly tied to the paragraph’s content.
* Return the result with the question as a valid JSON** in the form:

```json
{
  "question": "your question here"
}
```

Example:
User input:

```json
{
"title": "The Moon Landing",
"paragraph": "On July 20, 1969, Neil Armstrong became the first human to set foot on the Moon, followed by Buzz Aldrin."
}
```
Assistant output:

```json
{
  "question": "Who was the first human to set foot on the Moon during the Apollo 11 mission?"
}
```
"""

<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 4:</br>
In this system prompt, we don't generate answers, only questions. Explain why it's necessary in the context of white-box distillation.
<hr style="border:10px solid red"> </hr>  
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 4: </b><br>
Your answer here.
<hr style="border:10px solid green"> </hr>
</font></h4>

Here, we created a question generator. This, because we want to have a data/question dataset that are the same for the teacher and student. The teacher will then take this couple as an input and we will store the teacher data across the layers generated during the forward pass. Finally, giving the same data/question to the student, we will make it learn the teacher parameters from it.

<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 5:</br>
For synthetic data generation we'll be using vLLM.
vLLM is an optimized llm inference engine that can improve generation speed thanks to hardware specific optimization and computational tricks such as Prefix KV Caching.
(https://docs.vllm.ai/en/latest/features/automatic_prefix_caching.html)</br></br> 1. Explain why prefix caching will be very efficient in our case? </br></br>
2. What sampling `temperature` should we use? Justify.



<hr style="border:10px solid red"> </hr>  
</font></h4>


<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 5: </b><br>
Your answer here.
<hr style="border:10px solid green"> </hr>
</font></h4>

### **1. Explain why prefix caching will be very efficient in our case?**

Here, the question generator will generate questions from wikipedia articles from the same system prompt (its instructions). As this system prompt is always the same, and quite long, we do not want to compute attention from it every time as we will always have the sams values. Prefix caching allows us to cache the Key-Value matrices of this prompt. Then, we don't have to compute it for every steps, allowing us some real save in term of computational speed.


### **2. What sampling `temperature` should we use? Justify.**
We may use a temperature close to 0 in order to have what the teacher model will reponds in general. The higher is the temperature, the more we have the chance to have an atypical answer from the teacher model. Nevertheless, we don't necessarily want original answers here. We are more focus about transfering the general knowledge of the teacher to the student. A high temperature would create noise in the training and we don't want that.  

For distillation, we want:

- **Deterministic**, high-quality answers from the teacher that represent its "best" knowledge.
- **Consistent behavior**. The teacher should give its most confident, accurate responses.
- **Clear signal** for the student.

In order to have all of this, **I will suggest a temperature around 0.2**. Not 0 because in that case, the teacher answers might be too specific and we have a risk of overfitting.

<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 1: Optimized Generation of Synthetic Questions

Complete the below code with the adequate options (prefix caching and `temperature`)

<hr style="border:10px solid blue"> </hr>
</font></h4>


In [ ]:
from vllm import LLM
from vllm import SamplingParams
import os
path_teacher = "Qwen/Qwen2.5-7B-Instruct-AWQ"
llm = LLM(model=path_teacher, gpu_memory_utilization=0.7, max_model_len=5000, enable_prefix_caching=True)
sampling_params = SamplingParams(temperature = 0.2, max_tokens=400)


INFO 11-23 22:37:49 [utils.py:253] non-default args: {'max_model_len': 5000, 'enable_prefix_caching': True, 'gpu_memory_utilization': 0.7, 'disable_log_stats': True, 'model': 'Qwen/Qwen2.5-7B-Instruct-AWQ'}


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


INFO 11-23 22:37:51 [model.py:631] Resolved architecture: Qwen2ForCausalLM
INFO 11-23 22:37:51 [model.py:1745] Using max model len 5000
INFO 11-23 22:37:55 [scheduler.py:216] Chunked prefill is enabled with max_num_batched_tokens=8192.


Parse safetensors files:   0%|          | 0/2 [00:00<?, ?it/s]

WARNING 11-23 22:37:57 [system_utils.py:103] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized
INFO 11-23 22:39:07 [llm.py:352] Supported tasks: ['generate']


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 2: Question Generation</br>

<hr style="border:10px solid blue"> </hr>
</font></h4>

We're going to use the llm.chat vLLM api to generate our samples.
Make the adequate code to generate the data and save it in a questions.jsonl file.

Entries of jsonl file should look like:
```json
{
  "id_doc": <wikipedia_article_id>,
  "id_paragraph": <paragraph_dataset_id>,
  "question": <generated_question>,
  "title": <title_wikipedia_article>,
  "paragraph": <text_of_paragraph>
}
```

You should:

1. Complete `def extract_question(generated_text: str) -> str:` to extract the generated question.
2. Complete `def conversation_generator` to output a generator directly ingestible by `llm.chat` api
3. Complete the dataloader and for loop to generate samples by batches of 4
4. Save the generated questions in a `questions.jsonl` file

In [ ]:
from typing import Iterable, Iterator, List
import json
def extract_question(generated_text: str) -> str:

    """Extract the question from the generated text. If the question is not\
    following the right format return None."""

    try:
      j: dict
      j = json.loads(generated_text)
      assert "question" in j
      return j["question"]
    except (AssertionError, json.JSONDecodeError):
      return None


def conversation_generator(
    entries: Iterable[dict],
    system_prompt: str
    ) -> Iterator[dict]:

    """Generate the conversation with the model."""

    for entry in entries:
      conversation: List[dict]
      conversation = [
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": json.dumps({"title": entry["title"], "id_doc": entry["id"], "paragraph": entry["paragraph"]})}
      ]
      yield conversation

In [ ]:
from torch.utils.data import DataLoader
from datasets import load_dataset

ds = load_dataset("EvanD/Lab4_wikiparagraphs")

batch_size = 4
conversations = list(conversation_generator(ds["train"], system_prompt))

dataloader = DataLoader(conversations, batch_size=batch_size, shuffle=False, num_workers=2, prefetch_factor=2, collate_fn=lambda x: x)

In [ ]:
ds['train']['id']

In [ ]:
conversations[0]

In [ ]:
from tqdm.auto import tqdm
import jsonlines


with jsonlines.open("questions.jsonl", "w") as writer:
    for i, batch in tqdm(enumerate(dataloader)):
        id_paragraph = i * batch_size
        outputs = llm.chat(batch,
                        sampling_params=sampling_params,
                        use_tqdm=False)
        for j, output in enumerate(outputs):
          entry = ds['train'][id_paragraph + j]
          q = extract_question(output.outputs[0].text)

          if q:
            writer.write({
                "id_doc": entry["id"],
                "id_paragraph": id_paragraph,
                "question": q,
                "title": entry["title"],
                "paragraph": entry["paragraph"]
            })

0it [00:00, ?it/s]

INFO 11-23 22:39:13 [chat_utils.py:557] Detected the chat template content format to be 'string'. You can set `--chat-template-content-format` to override this.


In [ ]:
ds['train'][0]["paragraph"]

## 1.2 - Logprobs Generation

We get to the second part of this distillation where we are interested in distilling the answers logprobs of our teacher model to specialize our 0.5B model to perform Retrieval Augmented Generation (RAG).

First we're going to generate the logprobs with our 7B parameter model.

We'll use the following system prompt:

In [ ]:
system_prompt = (
    "You are an assistant for a Retrieval-Augmented Generation (RAG) system.\n"
    "Answer the question using only the provided documents. "
    "If the answer cannot be found in the provided documents, respond that the answer is not available in the provided document database. "
    "Documents:\n{context_block}\n\n"
)

<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 3: Complete Conversation Generator</br>

<hr style="border:10px solid blue"> </hr>
</font></h4>

Based on the previous code you made to generate questions, make a second one to generate answers and saving logprobs. We updated the sampling parameters of vLLM to return the logprobs and the token ids of the 20 most probable tokens.

As this can result in high quantity of data in practice we're going to store generated conversations in a .jsonl file and generated logprobs and token ids to a hdf5 file (see https://docs.h5py.org/en/stable/ for more information on hdf5).

`save_logprobs_hdf5()` is already implemented for you, and allows to save the logprobs to a .h5 hdf5 file, and increments sequences automatically

The structure of the conversation generator `conversation_generator()` function is implemented, you need to complete it.



In [ ]:
import h5py, os
import numpy as np

def save_logprobs_hdf5(path, sequences, start_idx=None):
    """
    sequences: list of sequences
       each sequence = list of steps
          each step = {token_id: Logprob(logprob=..., ...), ...}
    """
    mode = "a" if os.path.exists(path) else "w"
    with h5py.File(path, mode) as f:
        # choose index to start writing
        if start_idx is None:
            # auto-continue numbering if file already has data
            existing = [int(k.split("_")[1]) for k in f.keys() if k.startswith("seq_")]
            start_idx = max(existing)+1 if existing else 0

        for s_i, seq in enumerate(sequences, start=start_idx):
            g = f.create_group(f"seq_{s_i}")
            for t_i, step in enumerate(seq):
                token_ids = np.fromiter(step.keys(), dtype=np.int32)
                logprobs  = np.array([lp.logprob for lp in step.values()], dtype=np.float32)
                g.create_dataset(f"step_{t_i}/token_ids", data=token_ids, compression="gzip")
                g.create_dataset(f"step_{t_i}/logprobs",  data=logprobs,  compression="gzip")





<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 6:</br>
When completing conversation_generator we have several ways of sampling.
What are good paragraph sampling strategies we could use to ensure good performance of downstream model?

<hr style="border:10px solid red"> </hr>  
</font></h4>


<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 6: </b><br>
Your answer here.
<hr style="border:10px solid green"> </hr>
</font></h4>

In [ ]:
import random

doc_id_to_paragraphs = {}

for line in ds["train"]:
    doc_id = line["id"]
    paragraph = line["paragraph"]
    title = line["title"]
    if doc_id not in doc_id_to_paragraphs:
        doc_id_to_paragraphs[doc_id] = []
    doc_id_to_paragraphs[doc_id].append(title + " -- " + paragraph)  # We add the title to contextualize the paragraph

def conversation_generator(
    path_jsonl: str,
    system_prompt: str,
    top_k: int = 3
    ) -> Iterator[dict]:

    """Generate the conversation with the model."""

    with jsonlines.open(path_jsonl, "r") as f:
      for q_p in f:
        number_of_paragraphs_in_context = random.sample(range(1, top_k + 1), 1)[0]
        paragraphs = [q_p["title"] + " -- " + q_p["paragraph"]] # We add the title to contextualize the paragraph
        while len(paragraphs) < number_of_paragraphs_in_context:
          # With 50% probability, add a paragraph from the same article
          if random.random() < 0.5 and q_p["id"] in doc_id_to_paragraphs:
            same_doc_paragraphs = doc_id_to_paragraphs[q_p["id"]]
            candidates = [p for p in same_doc_paragraphs if p != q_p["title"] + " -- " + q_p["paragraph"]]
            if candidates:
                paragraphs.append(random.choice(candidates))
                continue

        # Add a paragraph from a different article
        different_doc_id = random.choice([doc_id for doc_id in doc_id_to_paragraphs.keys() if doc_id != q_p["id"]])
        paragraphs.append(random.choice(doc_id_to_paragraphs[different_doc_id]))



        random.shuffle(paragraphs)
        system_prompt_formatted = system_prompt.format(
            context_block="\n\n".join(f"[Document {i+1}]: {doc}" for i, doc in enumerate(paragraphs))
        )
        conversation = [
            {"role": "system", "content": system_prompt_formatted},
            {"role": "user", "content": q["question"]}
        ]
        yield conversation



In [ ]:
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from typing import Iterable


batch_size = 4

conversations = list(conversation_generator(path_jsonl="questions.jsonl", system_prompt=system_prompt)) # To Complete

dataloader = DataLoader(conversations, batch_size=batch_size, shuffle=False, num_workers=2, prefetch_factor=2, collate_fn=lambda x: x)

sampling_params = SamplingParams(temperature=0.2, max_tokens=400, logprobs=20)

with jsonlines.open("conversations_rag.jsonl", "w") as writer:
    for batch in tqdm(dataloader):
        out = llm.chat(batch,
                        sampling_params=sampling_params,
                        use_tqdm=False)
        for i, b in enumerate(batch):
            text = out[i].outputs[0].text
            b.append({"role": "assistant", "content": text})
            writer.write(b)
        save_logprobs_hdf5("logprobs.h5", [out[i].outputs[0].logprobs for i in range(len(out))])

'''
conversations = list(conversation_generator(path_jsonl="questions.jsonl", system_prompt=system_prompt))
'''

## 1.3 - KL-Divergence and Distillation

**You should restart the notebook kernel to free the gpu memory from the 7B model that is no longer needed**

Now that we have the generated conversations and their logprobs we can train our 0.5B model to output the same distribution.

The attentive student would have noticed that we have an incomplete representation of the probability distribution over tokens due to only keeping the top-20 logprobs.


<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 7:</br>
What are the two solutions you can see to approximate full distillation despite only having the top-20 logprobs?

<hr style="border:10px solid red"> </hr>  
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 7: </b><br>
Your answer here.
<hr style="border:10px solid green"> </hr>
</font></h4>

The issue is that limiting the teacher's output distribution to only the top-20 most probable tokens means we are ignoring the small, non-zero probabilities for the rest of the vocabulary.


Here are two solutions to approximate full distillation despite this limitation:


### **1. Zero-Out Missing Probabilities**:


Treat all tokens outside the teacher's top-20 list for a given time step as having a probability of 0 and log-probability of log(0)=−∞ (or the minimum finite value). We then have a simplified KL divergence calculation by only summing over the tokens in the top-20 list.


### **2. Adding a Uniform Probability Mass to the Tail**:


Estimate the residual probability mass as a uniform law on the rest of the tokens. Distribute this residual mass uniformly among the remaining tokens in the vocabulary. This provides a non-zero, consistent soft target for the tail tokens, making the KL divergence calculation more accurate.

We supply two functions to help in this implementation:

- `find_subsequence()`that allows to find all the occurences of a token subsequence in a 1-D tensor
- `get_labels()` that allows to expand the top-20 logprobs to the whole vocabulary, implicitly setting prob to zero for other tokens
- `QwenKLDataset`that loads samples from .h5 and .jsonl files, remove problematic inconsistent tokenized examples and outputs samples tokenized for training.

To simplify we will train with a batch size of 1, you can implement gradient accumulation if you wish.

The `PREFIX_ASSISTANT`variable contains the tokens that encode for

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")

In [ ]:
import torch

def find_subsequence(input_ids: torch.Tensor, subseq: torch.Tensor):
    """Find the first index of each occurence of a subsequence in the input_ids."""

    subseq_len = len(subseq)
    matches = []
    for idx in range(input_ids.size(0) - subseq_len + 1):
        if torch.equal(input_ids[idx:idx + subseq_len], subseq):
            return [idx]
    return []

def get_labels(logprobs: torch.Tensor, tokens, size_vocab: int = 151936, offset: int = 0):
    """Get the greedy max probability tokenized sequence from the logprobs."""
    labels = torch.full((len(logprobs), size_vocab), torch.finfo(torch.float16).min, dtype=torch.float16)
    for idx, logprobs_token in enumerate(logprobs):
        for token_id, logprob in zip(tokens[idx], logprobs_token):
            labels[idx][token_id] = logprob
    return labels

In [ ]:
from torch.utils.data import Dataset
import jsonlines
import h5py


PREFIX_ASSISTANT = [198, 151644, 77091, 198]

class QwenKLDataset(Dataset):
    """Dataset for finetuning Qwen model with KL divergence loss."""
    def __init__(
            self,
            path_h5,
            path_jsonl
        ):
        self.entries = []
        with jsonlines.open(path_jsonl, "r") as reader:
            with h5py.File(path_h5, "r") as f:
                for j, line in tqdm(enumerate(reader)):
                    inputs = tokenizer.apply_chat_template(line, add_generation_prompt=False, tokenize=True, return_dict=True, return_tensors="pt")
                    idx_subseq = find_subsequence(inputs["input_ids"][0], subseq = torch.tensor(PREFIX_ASSISTANT))[0]
                    seq_f = [f[f"seq_{j}"][f"step_{i}"]["token_ids"][0] for i in range(len(f[f"seq_{j}"]))]
                    try:
                        assert len(inputs["input_ids"][0][idx_subseq+3:-2]) == len(seq_f)
                    except AssertionError:
                        print(f"AssertionError {j}, skipping inconsistent detokenization/tokenization")
                    tokens = [f[f"seq_{j}"][f"step_{i}"]["token_ids"][:] for i in range(len(f[f"seq_{j}"]))]
                    logprobs = [f[f"seq_{j}"][f"step_{i}"]["logprobs"][:] for i in range(len(f[f"seq_{j}"]))]
                    inputs["input_ids"] = inputs["input_ids"].cuda()
                    self.entries.append(
                        {
                            "inputs": inputs,
                            "idx_subseq": idx_subseq + 3,
                            "seq_len": len(seq_f),
                            "logprobs": torch.tensor(np.array(logprobs)),
                            "tokens": torch.tensor(np.array(tokens))
                        }
                    )

    def __len__(self):
        return len(self.entries)

    def __getitem__(self, idx):
        """
        Return for a given entry:

        new_tokens: torch.Tensor, the tokenized conversation with the logprobs of the assistant answer inserted.
        idx_match: int, the index at which the assistant answer starts in the new_tokens.
        len_logprob_sequence: int, the tokenized length of the assistant answer.
        labels: torch.Tensor, the logprobs of the assistant answer for each token.
        """
        entry = self.entries[idx]
        entry["labels"] = get_labels(entry["logprobs"], entry["tokens"], offset=0, size_vocab=151936).cuda()

        return entry

KL-Divergence is a metric often used to quantify the difference of probability mass between two distributions.
It's not mathematically defined as a distance because of its asymetric nature:

$$D_{KL}(P \parallel Q) = \sum_{x} P(x) \log \frac{P(x)}{Q(x)}$$

In knowledge distillation, we often minimize the **Kullback–Leibler divergence** between the teacher’s output distribution $P_T$ and the student’s output distribution $P_S$.

$$
D_{KL}(P_T \parallel P_S) = \sum_x P_T(x) \log \frac{P_T(x)}{P_S(x)}
$$
$$
D_{KL}(P_S \parallel P_T) = \sum_x P_S(x) \log \frac{P_S(x)}{P_T(x)}
$$


<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 8:</br>
What qualitative difference would it make if we minimize $D_{KL}(P_T \parallel P_S)$ instead of $D_{KL}(P_S \parallel P_T)$? How would it affect the convergence of the student distribution?

<hr style="border:10px solid red"> </hr>  
</font></h4>



<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 8: </b><br>
Your answer here.
<hr style="border:10px solid green"> </hr>
</font></h4>


**DKL(PS∥PT)** :

It is also nammed "Mode Covering" in the litterature. When PS has probability but PT doesn't → high penalty. When PT has probability but PS doesn't → low penalty

Behavior: Student tries to cover all modes where teacher has probability. The student spreads its probability mass to avoid missing anything the teacher considers likely. Student distribution is often more diffuse/broad than teacher, covering multiple possible outputs even if some are low probability.


**DKL(PT∥PS)** :

Also nammed "Mode Seeking" in the litterature. When PT has probability but PS doesn't → high penalty. When PS has probability but PT doesn't → low penalty

Behavior: Student tries to focus on the main modes of the teacher. The student concentrates probability mass on the teacher's highest-probability regions.
Result: Student distribution is often more peaked/sharp, focusing on the teacher's most confident predictions and ignoring low-probability alternatives.
Effect on Convergence

**For distillation**, we will prefer to use DKL(PS∥PT) as we want the student to capture the full richness of the teacher's knowledge, not just its most confident predictions.

**You need to ensure you restarted the kernel and have sufficiently free memory (no 7B model running)**

check with `! nvidia-smi`

In [ ]:
! nvidia-smi




This code is very basic, and allows you to test a training over the small number of samples you've generated. Due to limitations of Google Colab we cannot go much further, but this first part should have given you the basics on how to perform white-box distillation.

<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 4: Complete The Training Code</br>

<hr style="border:10px solid blue"> </hr>
</font></h4>


- Loss function
- gradient accumulation handling



In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import get_linear_schedule_with_warmup
from tqdm.auto import tqdm
import torch
from torch.utils.data import DataLoader
import numpy as np


student_model = None # To Complete
dataset = None # To Complete

num_epochs = 2
grad_accum_steps = 8
optimizer = torch.optim.AdamW(student_model.parameters(), lr=5e-5)
loader = DataLoader(dataset, batch_size=1, shuffle=True, collate_fn=lambda x: x[0])
num_training_steps = num_epochs * len(loader) // grad_accum_steps
scheduler = get_linear_schedule_with_warmup(optimizer, 0, num_training_steps)
loss_fn = None # To Complete

for epoch in range(num_epochs):
    for i, batch in enumerate(loader):
        outputs = student_model(batch["inputs"]["input_ids"])
        # To Complete
        if (i + 1) % grad_accum_steps == 0 or (i + 1) == len(loader):
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            print (f"Epoch {epoch+1}, step {i+1}/{len(loader)}: loss = {batch_loss.item():.4f}")
    print(f"Epoch {epoch+1}: loss = {outputs.loss.item():.4f}")

student_model.save_pretrained("finetuned_model")
tokenizer.save_pretrained("finetuned_model")


# Part 2 - Retrieval Augmented Generation (RAG)

In this section, we will discuss the concept of **Retrieval-Augmented Generation (RAG)** — a framework that combines **information retrieval** and **language generation**. RAG enables language models to access **external knowledge sources** at inference time, reducing hallucinations and improving factual accuracy.

We will explore how to:
- Build and index a **Vector database** from a corpus (here: Wikipedia sample).
- Retrieve the most relevant documents given a query using **embedding-based similarity**.
- Integrate retrieval results into the **generation pipeline** to produce context-aware answers.


In [ ]:
# restart the session and run
!pip -q install chromadb==0.4.22
!pip -q install "numpy<2.0" --force-reinstall
!pip -q install datasets==2.21.0 pandas==2.2.2

In [ ]:
import os
import json
import random
import torch
import pandas as pd
import torch.nn.functional as F
from tqdm import tqdm
from datasets import load_dataset
from IPython.display import display, HTML
from transformers import AutoTokenizer, AutoModel

In [ ]:
# --- Configuration ---
SEED = 42
NUM_ROWS = 1000
DATA_PATH = "wikipedia_20231101_en_1000.csv"

if os.path.exists(DATA_PATH):
    print(f"✅ Found existing dataset at {DATA_PATH}")
    df = pd.read_csv(DATA_PATH)
else:
    print("⏳ Generating new dataset from Wikimedia (English, 2023-11-01)...")
    random.seed(SEED)

    # Load the Wikipedia dataset
    stream_ds = load_dataset(
        "wikimedia/wikipedia",
        "20231101.en",
        split="train",
        streaming=True
    )

    buffered_stream = stream_ds.shuffle(seed=SEED, buffer_size=200_000)

    sampled = []
    for ex in buffered_stream:
        try:
            if int(ex["id"]) % 2 == 0:
                sampled.append(ex)
            if len(sampled) >= NUM_ROWS:
                break
        except:
            continue

    # Create DataFrame
    df = pd.DataFrame(sampled)[["id", "url", "title", "text"]]
    df.to_csv(DATA_PATH, index=False)
    print(f"💾 Dataset saved to {DATA_PATH}")


In [ ]:
#Display Basic Info ---
print("Sampled shape:", df.shape)
print("Columns:", df.columns.tolist())
display(df.head())

In [ ]:
# Visualize Random Wikipedia Articles ---

NUM_EXAMPLES = 3  # number of random samples to show
samples = df.sample(NUM_EXAMPLES, random_state=random.randint(0, 10000))

for _, row in samples.iterrows():
    display(HTML(f"""
    <hr style="border:2px solid #ccc">
    <h3><b>Title:</b> {row['title']}</h3>
    <p><b>URL:</b> <a href="{row['url']}" target="_blank">{row['url']}</a></p>
    <p style="text-align: justify;"><b>Text:</b><br>{row['text']}</p>
    """))


### **Document Chunking**

The first step in building a RAG pipeline is **chunking**, where large documents are divided into smaller, semantically coherent pieces.  
Chunking allows the retriever to work on manageable text segments instead of entire documents, improving retrieval precision and reducing computational load.  


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 1: Naive Fixed-Length Chunking  
Split each document into overlapping fixed-length chunks to prepare text for retrieval.
<hr style="border:10px solid blue"> </hr>
</font></h4>


In [ ]:
def text_splitting(text, chunk_length=300, chunk_overlap=100):
    """
    Splits text into fixed-length chunks with overlap.
    """
    out = []
    stride = chunk_length - chunk_overlap ## FILL THE GAP: define the stride as the effective step between chunks
    for i in range(0, len(text), stride):
        chunk = text[i:i + chunk_length] ## FILL THE GAP: extract a substring of size 'chunk_length' starting at 'i'
        out.append(chunk)
    return out

# Apply to all documents
df["naive_chunks"] = df["text"].apply(lambda t: text_splitting(t, chunk_length=300, chunk_overlap=100))

num_chunks = df["naive_chunks"].apply(len)
print(f"Average number of chunks per document: {num_chunks.mean():.2f}")
print(f"Total number of chunks: {num_chunks.sum()}")

example_idx = 0
print("\n--- Example document ---")
print("Title:", df.iloc[example_idx]["title"])
print("Original length:", len(df.iloc[example_idx]["text"]))
print("Number of chunks:", len(df.iloc[example_idx]["naive_chunks"]))
print("\nFirst 2 chunks:\n")
for i, c in enumerate(df.iloc[example_idx]["naive_chunks"][:2]):
    print(f"Chunk {i+1}:\n{c[:400]}\n{'-'*80}")

<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 2: Paragraph-Aware Chunking  
Implement a smarter chunking strategy by using the ('.') as a boundary to split text into sentences or short paragraphs, then regroup them until reaching the desired chunk length.
<hr style="border:10px solid blue"> </hr>
</font></h4>


In [ ]:
def text_splitting_paragraph(text, chunk_length=300):
    """
    Splits text by sentences/paragraphs (using '.' as boundary)
    and groups them until reaching the desired chunk length.
    """
    out = []
    paragraph_list = text.split('. ')## FILL THE GAP: split the text into smaller parts using '.' as a separator
    current_text = ""
    length = 0
    for par in paragraph_list:
        if length != 0 and length + len(par) < chunk_length:
            current_text += '. ' + par ## FILL THE GAP: extend the ongoing chunk with the next segment
            length += len(par) ## FILL THE GAP: increment the total length accordingly
        else:
            if len(current_text) != 0:
                out.append(current_text) ## FILL THE GAP: store the current completed chunk before starting a new one)
            current_text = par ## FILL THE GAP: initialize a new chunk with the current paragraph
            length = len(current_text) ## FILL THE GAP: reset the chunk length counter
    if len(current_text) > 0:
        out.append(current_text) ## FILL THE GAP: add the last remaining chunk to the list)
    return out


# Apply to all documents
df["paragraph_chunks"] = df["text"].apply(lambda t: text_splitting_paragraph(t, chunk_length=300))

# Compute stats
num_chunks_par = df["paragraph_chunks"].apply(len)
print(f"Average number of paragraph-based chunks per document: {num_chunks_par.mean():.2f}")
print(f"Total number of paragraph-based chunks: {num_chunks_par.sum()}")

# Example comparison
example_idx = 432 #Check out other examples
print("\n--- Example document ---")
print("Title:", df.iloc[example_idx]['title'])
print("Original length:", len(df.iloc[example_idx]['text']))
print(f"Character based chunks: {len(df.iloc[example_idx]['naive_chunks'])}")
print(f"Paragraph based chunks: {len(df.iloc[example_idx]['paragraph_chunks'])}")

print("\nParagraph chunk preview:\n")
for i, c in enumerate(df.iloc[example_idx]['paragraph_chunks'][:6]):
    print(f"Chunk {i+1}:\n{c[:400]}\n{'-'*80}")


<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 1:  
In the paragraph-aware chunking method above, we simply split Wikipedia text using the '.' delimiter to approximate sentence boundaries.  
Discuss whether this is an effective strategy for creating meaningful chunks in a RAG system.
Propose one or more improved chunking strategies that could better capture document structure — and you may include code snippets to justify or demonstrate your approach.  
<hr style="border:10px solid red"> </hr>  
</font></h4>


<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 1: </b><br>
Your answer here.
<hr style="border:10px solid green"> </hr>
</font></h4>

 **Limitations:**
Splitting text using the "." delimiter to approximate sentence boundaries is a naive approach and has several limitations in the context of a RAG system. Indeed, not all periods "." indicate the end of a sentence. Abbreviations (e.g., "Dr.", "etc.") and decimal numbers can lead to incorrect splits, resulting in fragmented chunks. We also loose context when using the "." delimiter. It can break logical flows, especially in complex or technical texts. Sentences are often contextually linked.

 **Alternative:**
Instead of splitting only at ., split according to a fix chunk size or at paragraph boundaries (using \n\n tags for example) and ensure chunks have a small overlap to maintain context and this small overlap is a hyperparameter that we can tune.

<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 2: Consider a scenario where you want to perform RAG on source code (e.g., Python files, Java classes) instead of natural language text. Would the chunking methods demonstrated above (character-based and sentence/paragraph based with boundaries) work effectively for code? Explain why or why not, and describe how you would approach chunking source code to maintain semantic coherence. What specific characteristics of code structure would you need to consider?

<hr style="border:10px solid red"> </hr>
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 2: </b><br>
Your answer here.
<hr style="border:10px solid green"> </hr>
</font></h4>

The chunking method demonstrated above is efficient only for character-based and sentence/paragraph based with boundaries, not effective for chunking source code. Chunking source code requires language aware parsing and an understanding of code structure. Unlike natural language, code cannot be split arbitrarily chunks must be syntactically valid and semantically coherent. We can identify logical blocks (functions, classes, loops), then chunk at block boundaries to ensure syntactic completeness, also add overlaps for shared variables or logic and finally include imports in the first chunk or as needed.

In [ ]:
# Saving chunks ---

# Flatten chunks into a new DataFrame
records = []
for _, row in df.iterrows():
    doc_id = row["id"]
    title = row["title"]
    url = row["url"]
    for i, chunk in enumerate(row["paragraph_chunks"]):
        records.append({
            "doc_id": doc_id,
            "title": title,
            "url": url,
            "chunk_id": f"{doc_id}_chunk_{i}",
            "chunk_text": chunk.strip()
        })

# Create the flattened chunks DataFrame
chunks_df = pd.DataFrame(records)
print(f"Total chunks: {len(chunks_df)}")
print(f"Average chunk length: {chunks_df['chunk_text'].apply(len).mean():.2f} characters\n")

# Show example
print("Example rows:")
display(chunks_df.head())

chunks_df.to_csv("wikipedia_chunks.csv", index=False)
print("Chunks saved to 'wikipedia_chunks.csv'")
print(f"Total chunks: {len(chunks_df)}")


## <b>Part II: Embedding</b>

After chunking our documents, the next step is to convert text chunks into vector representations (embeddings). These embeddings capture the semantic meaning of the text in a high-dimensional space, allowing us to measure similarity between chunks and queries mathematically.

We will use **sentence-transformers/all-MiniLM-L6-v2**, a compact and efficient embedding model that produces 384-dimensional embeddings for English text. This model offers a strong balance between performance and computational efficiency, making it well-suited for our RAG pipeline.


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 3: </b><br>
Fill in the <code>embed()</code> function to encode text chunks and generate normalized embeddings using <code>sentence-transformers/all-MiniLM-L6-v2</code>.  
Then, apply it to all documents in <code>chunks_df</code> and store the results.
<hr style="border:10px solid blue"> </hr>
</font></h4>


In [ ]:
# --- 2.3 Embedding Generation ---
# Load model
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()

print(f"Loaded embedding model: {model_name}")


# --- Define embedding function ---
def embed(text_list, doc_type="document"):
    """
    Encodes a list of texts and returns normalized embeddings.
    """
    encoded = tokenizer(
        [f"search_{doc_type}: {t}" for t in text_list],
        padding=True,
        truncation=True,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        output = model(**encoded) ## FILL THE GAP: forward pass through the model to obtain hidden states
        token_embeddings = output.last_hidden_state ## FILL THE GAP: extract the last hidden state from the output
        pooled = torch.mean(token_embeddings, dim=1) ## FILL THE GAP: aggregate token embeddings (e.g., by summing along sequence dimension)
        pooled = torch.nn.functional.normalize(pooled, p=2, dim=1) ## FILL THE GAP: apply L2 normalization along the embedding dimension
    return pooled.cpu()


In [ ]:
# --- Test with one example ---
sample_text = ["Artificial intelligence is transforming the world."]
sample_emb = embed(sample_text)
print(f"Sample embedding shape: {sample_emb.shape}")

# --- Apply to all chunks ---
print(f"\nGenerating embeddings for {len(chunks_df)} chunks...")

emb_list = []
for i in tqdm(range(0, len(chunks_df), 32)):
    batch = chunks_df["chunk_text"].iloc[i:i+32].tolist()
    emb = embed(batch, doc_type="document")
    emb_list.append(emb)

chunk_embeddings = torch.cat(emb_list, dim=0).numpy()
chunks_df["embedding"] = list(chunk_embeddings)

print("\nEmbeddings generated and added to DataFrame.")
print(chunks_df[["chunk_id", "title", "embedding"]].head())

<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 3: In most retrieval systems, embeddings are represented as fixed-size vectors. Can you think of a way to design embeddings that can flexibly adjust their size or level of detail while still preserving meaningful similarity between representations? How could such an approach benefit Retrieval-Augmented Generation (RAG) systems in practice, particularly for improving efficiency or adapting to different computational budgets?

<hr style="border:10px solid red"> </hr>
<i>Hint:</i> You can find the answer in the paper <a href="https://arxiv.org/pdf/2205.13147" target="_blank">Matryoshka Representation Learning</a>.
</font></h4>


In most retrieval systems, embeddings are represented as fixed-size vectors.
The Matryoshka Representation Learning (MRL) offers a solution for flexible embeddings by training models with a multi-scale loss function. It ensures truncated prefixes of the full embedding vector maintain meaningful semantic information at multiple granularities simultaneously.

The model is optimized such that the first m dimensions of a d-dimensional embedding each preserve useful similarity relationships. The most critical information is encoded in early dimensions and progressively finer details in later ones. For RAG systems, this enables cascaded retrieval where low-dimensional embeddings rapidly screen millions of candidates. It is then followed by higher-dimensional refinement on reduced sets, achieving 10-50× speedup while maintaining retrieval quality.

This approach provides dynamic resource allocation based on computational resources, and it also reduces storage costs in vector databases. It enables efficient deployment across devices with varying capabilities. This make RAG systems more scalable, efficient, and practical for real-world applications without sacrificing semantic fidelity.

### **Building a Simple Vector Database**

After generating embeddings for all our document chunks, the next step is to **store** them in a structure that allows fast similarity search.  
In this section, we will build a **simple in-memory vector database** using PyTorch tensors.  
Each entry in the database will correspond to a text chunk and its embedding, enabling efficient retrieval based on vector similarity.


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 4: </b><br>
Fill in the code to populate the database by computing embeddings for all text chunks in <code>chunks_df</code> using the <code>embed()</code> function.
<hr style="border:10px solid blue"> </hr>
</font></h4>


In [ ]:
def populate_database(chunks_df, batch_size=16):
    """
    Populates a vector database from precomputed chunks_df.
    """
    n_chunks = len(chunks_df)

    sample_emb = embed([chunks_df["chunk_text"].iloc[0]], doc_type="document")  ## FILL THE GAP: compute one sample embedding to infer the output dimension
    output_dim = sample_emb.shape[1] ## FILL THE GAP: extract the embedding dimension from the sample

    vectorial_database = torch.zeros((n_chunks, output_dim)) ## FILL THE GAP: initialize an empty tensor to store all embeddings
    chunk_list = chunks_df["chunk_text"].tolist()

    print(f"Populating vector database with {n_chunks} chunks...")

    n = 0
    for i in range(0, n_chunks, batch_size):
        batch = chunk_list[i:i + batch_size] ## FILL THE GAP: select a batch of chunk texts
        embeddings = embed(batch, doc_type="document") ## FILL THE GAP: compute embeddings for the current batch
        vectorial_database[n:n + len(batch)] = embeddings ## FILL THE GAP: store embeddings in the tensor
        n += len(batch)

    return chunk_list, vectorial_database

In [ ]:
#Build Vector Database
chunk_list, vectorial_database = populate_database(chunks_df)

print("\n✅ Vector database successfully built.")
print(f"Total stored chunks: {len(chunk_list)}")
print(f"Database tensor shape: {tuple(vectorial_database.shape)}")

In [ ]:
#Save vector databse
os.makedirs("vector_db", exist_ok=True)

# Save tensor + chunk list
torch.save(vectorial_database, "vector_db/vectorial_database.pth")

with open("vector_db/chunk_list.json", "w", encoding="utf-8") as f:
    json.dump(chunk_list, f, indent=4, ensure_ascii=False)

print("✅ Saved:")
print(" - vector_db/vectorial_database.pth")
print(" - vector_db/chunk_list.json")

In [ ]:
# Load the database
vectorial_database = torch.load("vector_db/vectorial_database.pth", map_location=device)
vectorial_database.requires_grad_(False)

with open("vector_db/chunk_list.json", "r", encoding="utf-8") as f:
    chunk_list = json.load(f)

print(f"✅ Loaded {len(chunk_list)} chunks.")
print(f"Database shape: {vectorial_database.shape}\n")

# Inspect first few entries
for i, embedding_vector in enumerate(vectorial_database[:5]):
    print(f"Vector {i} → {embedding_vector[:5]}")
    print(f"Text snippet: {chunk_list[i][:300]}\n")


#### **Defining Similarity Metrics for Retrieval**

After populating our vector database with embeddings, the next step in a RAG pipeline is to define a *similarity metric* to measure how close two vectors are in the embedding space.  
Common metrics include **dot product**, **L2 distance**, and **cosine similarity**.  

In most retrieval systems, cosine similarity is preferred because it measures the *angle* between two vectors rather than their magnitude, allowing comparison based purely on semantic direction instead of scale.


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 5: </b><br>
Fill in the code to implement the <code>cosine_similarity()</code> function.  
<hr style="border:10px solid blue"> </hr>
</font></h4>


In [ ]:
def cosine_similarity(query_embeddings, doc_embeddings):
    """
    Computes cosine similarity between query and document embeddings
    using manual normalization.
    """
    query_magnitudes = torch.norm(query_embeddings, dim=1, keepdim=True) ## FILL THE GAP: calculate vector length for each query
    normalized_queries = query_embeddings / query_magnitudes ## FILL THE GAP: normalize queries using their magnitudes

    doc_magnitudes = torch.norm(doc_embeddings, dim=1, keepdim=True) ## FILL THE GAP: calculate vector length for each document
    normalized_docs = doc_embeddings / doc_magnitudes ## FILL THE GAP: normalize documents using their magnitudes

    similarity_matrix = torch.mm(normalized_queries, normalized_docs.transpose(0, 1)) ## FILL THE GAP: perform dot product between normalized queries and documents

    return similarity_matrix


# --- Example test ---
query_embeddings = embed([
    "What is t-SNE?",
    "Who is Laurens van der Maaten?"
], "query")

doc_embeddings = embed([
    "t-SNE is a dimensionality reduction algorithm created by Laurens van der Maaten."
], "document")

with torch.no_grad():
    sim_cos = cosine_similarity(query_embeddings, doc_embeddings)

print("🔍 Example cosine similarity scores:\n", sim_cos)

<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 6: </b><br>
Fill in the code to complete the <code>retrieve()</code> function.  
It should encode the input query using <code>embed()</code>, compute similarity with all vectors in <code>vectorial_database</code>, and return the top-<i>k</i> most similar text chunks.
<hr style="border:10px solid blue"> </hr>
</font></h4>


In [ ]:
def retrieve(query,
             vectorial_database=vectorial_database,
             chunk_list=chunk_list,
             topk=5,
             verbose=False):
    """
    Retrieves top-k most similar chunks to a query using dot-product similarity.
    """
    with torch.no_grad():
        query_embedding = embed([query], doc_type="query")[0]  ## FILL THE GAP: encode the input query using the embed() function
        query_embedding = query_embedding.to(device) ## FILL THE GAP: move the query embedding to the correct device
        similarity_scores = torch.mm(
            query_embedding.unsqueeze(0),
            vectorial_database.t()
        ) ## FILL THE GAP: compute similarity between query and database embeddings
        topk_results = torch.topk(similarity_scores, k=topk) ## FILL THE GAP: extract the top-k highest similarity scores and indices

        if verbose:
            for score, idx in zip(topk_results.values[0], topk_results.indices[0]):
                print(f"\nScore: {score:.4f}")
                print(f"Chunk:\n{chunk_list[idx][:500]}\n{'-'*80}")

        retrieved_chunks = [chunk_list[idx] for idx in topk_results.indices[0]]   ## FILL THE GAP: select text chunks corresponding to the top-k indices
        return "\n\n".join(retrieved_chunks) ## FILL THE GAP: return concatenated retrieved chunks as a single string

In [ ]:
# Example query
query = "When was Luigi Boria born?" #Try different queries based on the documents in the wikipedia dataset
result = retrieve(query, topk=3, verbose=True)


<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr> Question 4: There are retrieval methods like BM25 that rely on lexical overlap between the query and documents, and others based on dense embeddings that capture semantic similarity beyond exact word matches. Explain how these two approaches differ in how they represent and compare text. Then, discuss how a hybrid retrieval strategy combining both can overcome their respective limitations and improve retrieval performance in RAG systems. <hr style="border:10px solid red"> </hr> </font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 4: </b><br>
Your answer here.
<hr style="border:10px solid green"> </hr>
</font></h4>


**BM25** uses sparse vectors based on exact word matching and term frequency statistics. It counts how often query words appear in documents, weighted by their rarity across the corpus. However, it fails to recognize synonyms "car" and "automobile" are treated as completely different terms. It cannot handle paraphrasing or understand semantic meaning beyond surface level word overlap.

**Dense embeddings** map text into continuous vector spaces using neural networks. They capture conceptual relationships and contextual meaning regardless of exact words used. They understand that "vehicle" and "car" are semantically related. But they may miss important exact keyword matches and struggle with rare technical terms not seen during training.

**Hybrid retrieval** combines both approaches for better performance. It retrieves candidates using both BM25 and dense embeddings separately. Results are merged using techniques like Reciprocal Rank Fusion, which combines rankings from both methods. This captures BM25's strength with specific keywords, proper nouns, and technical jargon.

**For RAG systems**, hybrid retrieval is particularly valuable because users ask diverse question types. Some queries need exact term matching. Others need semantic understanding. Hybrid approaches achieve 10-30% better recall and precision compared to using either method alone. They make RAG systems more robust and reliable across different query styles and document types.

In real-world RAG systems, instead of manually storing and comparing vectors, we rely on **vector databases** such as **ChromaDB**, which are optimized for efficient **similarity search**, **indexing**, and **retrieval** at scale.  

These databases provide:
- Fast nearest-neighbor search (e.g., using HNSW graphs)  
- Persistent storage for millions of embeddings  
- Built-in support for different similarity metrics (cosine, L2, inner product)  


In [ ]:
import chromadb
from chromadb.config import Settings

# --- Initialize ChromaDB client ---
chroma_client = chromadb.Client(Settings(
    anonymized_telemetry=False,
    allow_reset=True
))

# Reset ensures a clean state
chroma_client.reset()

# --- Create a collection ---
# You can choose the similarity metric: "cosine", "l2", or "ip" (inner product)
collection_name = "wikipedia_chunks"
collection = chroma_client.create_collection(
    name=collection_name,
    metadata={"hnsw:space": "cosine"}  # cosine similarity works best for normalized embeddings
)

print(f"✅ Created collection: {collection_name}")


In [ ]:
# Prepare the data
ids = chunks_df["chunk_id"].tolist()
embeddings = chunks_df["embedding"].tolist()
documents = chunks_df["chunk_text"].tolist()

# Ensure all embeddings are plain Python lists
embeddings = [e.tolist() if hasattr(e, "tolist") else e for e in embeddings]

# Add useful metadata for inspection
metadatas = [
    {
        "doc_id": row["doc_id"],
        "title": row["title"],
        "url": row["url"]
    }
    for _, row in chunks_df.iterrows()
]

# Add to the collection
collection.add(
    ids=ids,
    embeddings=embeddings,
    documents=documents,
    metadatas=metadatas
)

print(f"\n✅ Added {collection.count()} chunks to the collection")
print(f"Collection metadata: {collection.metadata}")


In [ ]:
# Encode the query using our embed() function
query = "When was Luigi Boria born?"
query_embedding = embed([query], doc_type="query")[0].tolist()  # get single vector as list

# Query the collection
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=3 # number of retrieved results
)

# Display results
print(f"🔎 Query: {query}\n" + "=" * 80)
for i in range(len(results["documents"][0])):
    print(f"\nResult {i+1}:")
    print(f"Title: {results['metadatas'][0][i]['title']}")
    print(f"Similarity score: {1 - results['distances'][0][i]:.4f}")  # cosine distance → similarity
    print(f"Text: {results['documents'][0][i][:300]}...")
    print("-" * 80)


<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 5: </b><br>
ChromaDB and other vector databases often rely on Hierarchical Navigable Small World (HNSW) graphs to perform efficient approximate nearest neighbor search.  
Explain how the HNSW algorithm organizes data to enable fast and accurate retrieval in high-dimensional spaces.  
Why is this structure particularly effective for large-scale embedding collections compared to brute-force search?

<hr style="border:10px solid red"> </hr>
<i>Reference:</i> <a href="https://arxiv.org/pdf/1603.09320" target="_blank">Efficient and Robust Approximate Nearest Neighbor Search using Hierarchical Navigable Small World Graphs</a>
</font></h4>


<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 5: </b><br>
Your answer here.
<hr style="border:10px solid green"> </hr>
</font></h4>

**HNSW** is a graph-based indexing structure designed for fast approximate nearest neighbor search in high-dimensional spaces. It builds on the concept of small-world networks where most nodes are reachable through a small number of hops.

**Core Structure**: HNSW organizes embeddings into multiple hierarchical layers. The top layers are sparse and contain long-range connections for rapid navigation. Bottom layers are dense with short-range connections for precise local search. Each data point appears in layer 0, but only some points propagate to higher layers with exponentially decreasing probability.

**Search Process**: Search starts at the topmost layer with a random entry point. It greedily navigates toward the query using long-range connections to quickly approach the target region. As it descends through layers, it refines the search with increasingly local connections. At layer 0, it performs a final greedy search among nearest candidates. This multi-scale approach mimics zooming from a global map view to street-level detail.

**Why It's Effective**: HNSW achieves search complexity O(log N) instead of linear O(N) in brute-force search. For a collection of 1 million embeddings, brute-force requires 1 million distance calculations. HNSW needs only hundreds. The hierarchical structure enables efficient pruning. Long-range connections in upper layers allow the algorithm to skip large irrelevant regions.

**For Vector Databases**: HNSW provides very recall value (>95%) while being 100-1000× faster than exhaustive search. It handles billion-scale collections that would be impractical with brute-force. Memory efficiency comes from storing only graph connections, not redundant data copies. This makes HNSW effective for RAG systems where embedding collections grow continuously and query latency must remain low.

### <b> Two-Stage Retrieval: Dense Retrieval + Reranker</b>

In RAG, the first retrieval step often returns passages that are similar in meaning but not always the most relevant.  
A **reranker** fixes this by re-evaluating the top retrieved chunks using a stronger model that jointly reads the query and each document to assign a more accurate relevance score.


In [ ]:
from sentence_transformers import CrossEncoder

# Load reranker model
RERANKER_MODEL_NAME = "cross-encoder/ms-marco-MiniLM-L-6-v2"
reranker = CrossEncoder(RERANKER_MODEL_NAME)

def retrieve_with_reranker(query, collection, initial_k=5, final_k=3):
    """
    Retrieves and reranks candidate documents for a given query.
    Returns both the initial dense results and reranked results.
    """
    query_embedding = embed([query], doc_type="query")[0].tolist()
    candidates = collection.query(query_embeddings=[query_embedding], n_results=initial_k)

    docs = candidates["documents"][0]
    metas = candidates["metadatas"][0]
    dense_scores = [(1 - s) for s in candidates["distances"][0]]

    pairs = [(query, d) for d in docs]
    ce_scores = reranker.predict(pairs)

    reranked = [
        {
            "title": metas[i].get("title", ""),
            "url": metas[i].get("url", ""),
            "text": docs[i],
            "dense_score": dense_scores[i],
            "rerank_score": float(ce_scores[i]),
        }
        for i in range(len(docs))
    ]
    reranked.sort(key=lambda x: x["rerank_score"], reverse=True)

    return docs, metas, dense_scores, reranked[:final_k]

In [ ]:
# Example usage
query = "Where did Luigi Boria study?" #Try other queries too
docs, metas, dense_scores, top_reranked = retrieve_with_reranker(
    query=query,
    collection=collection,
    initial_k=5,
    final_k=3
)

# --- Display initial dense retrieval ---
print(f"\nInitial dense retrieval (Top 5):\n" + "=" * 80)
for i, (d, s, m) in enumerate(zip(docs, dense_scores, metas), 1):
    print(f"{i}. {m.get('title', '')}  |  Dense similarity: {s:.4f}")
    print(f"Text: {d[:300].replace('\n', ' ')}")
    print("-" * 80)

# --- Display top reranked results ---
print(f"\nAfter Cross-Encoder Reranking (Top 3):\n" + "=" * 80)
for i, item in enumerate(top_reranked, 1):
    print(f"{i}. {item['title']}")
    print(f"Dense similarity: {item['dense_score']:.4f} | Reranker score: {item['rerank_score']:.4f}")
    print(f"Text: {item['text'][:300].replace('\n', ' ')}")
    print("-" * 80)

![Bi-Encoder vs Cross-Encoder Architecture](https://raw.githubusercontent.com/UKPLab/sentence-transformers/master/docs/img/Bi_vs_Cross-Encoder.png)

<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 6:  
The figure above compares a Bi-Encoder and a Cross-Encoder architecture.  
Rerankers such as <code>cross-encoder/ms-marco-MiniLM-L-6-v2</code> use the second approach, jointly encoding the query and document through a single transformer.  
Why does this joint encoding typically yield higher retrieval precision, and why is it applied as a second-stage reranker instead of being used directly for large-scale retrieval?
<hr style="border:10px solid red"> </hr>
</font></h4>


<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 6: </b><br>
Your answer here.
<hr style="border:10px solid green"> </hr>
</font></h4>

### Why Cross-Encoders Have Higher Precision

**Joint encoding allows richer interaction:**
- The Cross-Encoder processes query and document together through all transformer layers
- Attention mechanisms can compute token-level interactions between query and document from the start
- The model learns nuanced semantic relationships: which document tokens specifically answer which query tokens
- Results in much more accurate relevance scoring

**Bi-Encoder limitations:**
- Encodes query and document independently into separate embeddings
- Only compares them via simple cosine similarity at the end
- No direct attention between query and document tokens
- Misses fine-grained semantic matching

### Why Cross-Encoders Are Only Used for Reranking and not directly for large-scale retrieval

**Computational cost:**
- Cross-Encoder must process every (query, document) pair jointly
- For 1 query + 1 million documents = 1 million forward passes
- Does not scale to large document collections

**Bi-Encoder advantage for first-stage retrieval:**
- Encode all documents once into embeddings, store them
- At query time: encode query once, compute cosine similarity with pre-computed document embeddings
- Fast approximate nearest neighbor search (ANN) retrieves top-k candidates in milliseconds

## The Two-Stage Pipeline

1. **Bi-Encoder (retrieval)**: Fast, scalable - retrieve top 100-1000 candidates from millions
2. **Cross-Encoder (reranking)**: Slow, accurate - precisely rerank the small candidate set


### **Integrating Retrieved Context into the LLM’s Prompt**

Now that we can retrieve and rerank the most relevant document chunks,  
we integrate them directly into the **language model’s prompt**.  
This step allows the model to **ground its answer on factual context** rather than relying solely on internal knowledge —  
thereby improving accuracy and reducing hallucinations.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

gen_model_name = "Qwen/Qwen2.5-0.5B-Instruct"
gen_tok = AutoTokenizer.from_pretrained(gen_model_name, trust_remote_code=True)
gen_model = AutoModelForCausalLM.from_pretrained(
    gen_model_name,
    device_map="auto",
    dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    trust_remote_code=True
)

def generate(msg, max_new_tokens=128, temperature=0.2):
    messages = [{"role": "user", "content": msg}]
    inputs = gen_tok.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(gen_model.device)

    outputs = gen_model.generate(
        inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_p=0.9,
        do_sample=True,
        eos_token_id=gen_tok.eos_token_id,
        pad_token_id=gen_tok.pad_token_id if gen_tok.pad_token_id is not None else gen_tok.eos_token_id
    )

    return gen_tok.decode(outputs[0][inputs.shape[-1]:], skip_special_tokens=True).strip()


# ============================================================
# NO-RAG vs WITH RAG
# ============================================================
query = "When was Luigi Boria born?" #Try other queries

print("ORIGINAL PROMPT\n" + "=" * 60)
print(query)

print("\nANSWER WITHOUT RAG\n" + "=" * 60)
print(generate(query))

docs, metas, dense_scores, top_reranked = retrieve_with_reranker(
    query=query,
    collection=collection,
    initial_k=5,
    final_k=3
)

context = "\n\n".join(h["text"] for h in top_reranked)[:1600]
rag_prompt = (
    f"Use only the context to answer. If unknown, say you don't know.\n\n"
    f"Context:\n{context}\n\n"
    f"Question: {query}\nAnswer:"
)

print("\nAUGMENTED PROMPT\n" + "=" * 60)
print(rag_prompt)

print("\nANSWER WITH RAG\n" + "=" * 60)
print(generate(rag_prompt))


<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr> Question 7: In the RAG prompt construction step, we simply concatenate the retrieved chunks before the question. Discuss potential issues with this naive approach, such as token limits, redundancy, or irrelevant context dilution. Then, explain how we could select, weight, or summarize the retrieved chunks before injecting them into the prompt to improve generation quality and efficiency. <hr style="border:10px solid red"> </hr> </font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 7: </b><br>
Your answer here.
<hr style="border:10px solid green"> </hr>
</font></h4>

**Problems with Naive Concatenation:**

- Token limits: Long concatenated chunks exhaust context windows and increase costs.
- Redundancy: Multiple chunks repeat the same information, wasting tokens.
- Noise: Irrelevant chunks dilute important context and confuse the model ("lost in the middle" effect).

**Better Approaches:**
- Reranking: Use cross-encoders to score chunks and select only top 3-5 most relevant ones.
- Compression: Summarize chunks extractively or abstractively to reduce length while preserving key information.
- Weighting: Position most relevant chunks near the question where models attend better. Add relevance scores as metadata.

#### **To go further**

- Experiment with other chunking methods (e.g., semantic or recursive chunking).  
- Explore **LangChain** and **LlamaIndex** for building modular RAG pipelines.  
- Try **hybrid retrieval** combining sparse (BM25) and dense embeddings.  
- Explore more advanced RAG methods such as **RAG-Fusion**, **Self-RAG**, and **Active-RAG**.  
- Experiment with **Matryoshka Representation Learning** for scalable embeddings.  
- Try **fine-tuning rerankers** or **retrievers** for domain-specific data.  
